## Goal: Generate IGVF perturb-seq metadata information given input .fasta file with protospacer sequences

Requirements: https://docs.google.com/document/d/1Z1SOlekIE5uGyXW41XxnszxaYdSw0wdAOUVzfy3fj3M/edit?tab=t.0#heading=h.lw69v09vjkrr

Example file: /hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/example_perturbseq_seqfile.tsv

Note: I was having trouble with BLAT CLI so I used webtool to get most fields. Still need PAM and TSS coordinate


### Extract PAM

In [32]:
!module load Bedtools/2.30.0 

import pandas as pd
import os

def compute_upstream_with_pam(df, genome_fasta="/hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/perturb-seq_metadata/hg38.fa"):
    output_bed = "temp_upstream.bed"
    output_fasta = "temp_upstream.fa"

    upstream_data = []

    with open(output_bed, "w") as bed:
        for _, row in df.iterrows():
            chrom = row["guide_chr"]
            start = row["guide_start"]
            end = row["guide_end"]
            strand = row["strand"]
            guide_id = row["guide_id"]

            # Cas9 PAM (NGG) is on the 3' end of the guide
            if strand == "+":
                pam_start = end  # NGG is immediately downstream
                pam_end = end + 3  # Three bases downstream
            else:
                pam_start = start - 3  # Three bases upstream for reverse strand
                pam_end = start  

            # Write to BED file
            bed.write(f"{chrom}\t{pam_start}\t{pam_end}\t{guide_id}\t.\t{strand}\n")

    # Run bedtools getfasta to extract sequence
    bedtools_path = "/opt/apps/rhel8/bedtools-2.30.0/bin/bedtools"
    os.system(f"{bedtools_path} getfasta -fi {genome_fasta} -bed {output_bed} -fo {output_fasta} -s")

    # Read extracted sequences
    with open(output_fasta, "r") as fasta:
        sequences = fasta.read().splitlines()

    # Process FASTA output
    for i in range(0, len(sequences), 2):  # FASTA format is >header, seq
        guide_id = sequences[i].strip(">").split(":")[0]  # Extract guide_id from header
        pam_seq = sequences[i + 1]  # Extract PAM sequence

        upstream_data.append([guide_id, pam_seq])

    # Create DataFrame
    upstream_df = pd.DataFrame(upstream_data, columns=["guide_id", "PAM_sequence"])
    
    # Save output
    upstream_df.to_csv("upstream_regions_with_pam.csv", sep=",", index=False)
    return upstream_df

# Read input CSV
df = pd.read_csv("/hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/perturb-seq_metadata/PAM_input.csv", sep=",")  # Adjust separator if needed

# Compute upstream regions and extract PAM
upstream_df = compute_upstream_with_pam(df)

print(upstream_df.head())  # Show output


Bedtools 2.30.0
>

index file /hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/perturb-seq_metadata/hg38.fa.fai not found, generating...


Empty DataFrame
Columns: [guide_id, PAM_sequence]
Index: []


Unexpected file format.  Please use tab-delimited BED, GFF, or VCF. Perhaps you have non-integer starts or ends at line 1?


In [23]:
import os
print(os.getcwd())

/hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files


In [22]:
import csv
with open('/hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/grna_libs/SJR_CRISPRa_TF_lib_final.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    print("Detected fieldnames:", reader.fieldnames)
    for row in reader:
        print(row)
        break  # Just check the first row

Detected fieldnames: ['\ufeffgene', 'protospacer', 'oligo']
{'\ufeffgene': 'ADNP.1', 'protospacer': 'AACCCCCCCTGGGGAAAAGG', 'oligo': 'ATATATCTTGTGGAAAGGACGAAACACCGAACCCCCCCTGGGGAAAAGGGTTTAAGAGCTATGCTGGAAACAGCATAG'}


In [4]:
import csv
input_csv = '/work/rr151/SamAIM2neuro_data/subtype_lib.csv'
output_fasta = '/work/rr151/SamAIM2neuro_data/SJR_subtype_lib_final.fasta'
with open(input_csv, 'r', newline='') as csvfile, open(output_fasta, 'w') as fastafile:
    reader = csv.DictReader(csvfile)
    reader.fieldnames = [field.strip().replace('\ufeff', '') for field in reader.fieldnames]
    for row in reader:
        gene = row['gene'].strip()
        spacer = row['protospacer'].strip()
        fastafile.write(f'>{gene}\n{spacer}\n')

In [10]:
%%bash
module load Bowtie2/2.4.4-rhel8
#module load Bowtie/1.2.2
#bowtie2-build '/hpc/group/gersbachlab/Reference_Data/Genomes/hg38/hg38.fa' '/work/rr151/SamAIM2neuro_data/hg38_index'

Bowtie2 2.4.4-rhel8


In [12]:
#!/bin/bash
# Inputs
FASTA="/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final.fasta"
BOWTIE_INDEX="/work/rr151/SamAIM2neuro_data/hg38_index"
SAM_OUT="/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final.sam"
TSV_OUT="/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final_metadata.tsv"

#module load Bowtie2/2.4.4-rhel8
# Step 1: Align using Bowtie
#echo "Running Bowtie..."
#bowtie -f -v 0 -a --best --strata $BOWTIE_INDEX $FASTA > $SAM_OUT
#bowtie2 -f -x $BOWTIE_INDEX -U $FASTA -S $SAM_OUT
#bowtie2 -f -x /work/rr151/SamAIM2neuro_data/hg38_index -U SJR_subtype_lib_final.fasta -S SJR_subtype_lib_final.sam
# Step 2: Generate metadata
echo "Parsing SAM to metadata..."
#python3 /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_metadata.py $FASTA $SAM_OUT $TSV_OUT
python3 /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_metadata.py $SAM_OUT $TSV_OUT
echo "Done. Metadata written to $TSV_OUT"

SyntaxError: invalid syntax (<ipython-input-12-c898449e6a35>, line 14)

In [14]:
%%writefile /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_metadata.py
import sys
import csv
import requests
#from Bio import SeqIO
# Inputs from command line
#fasta_file = sys.argv[1]
sam_file = sys.argv[1]
output_file = sys.argv[2]

import csv
input_csv = '/hpc/group/gersbachlab/sjr72/IGVF_AtN_Submission_Files/grna_libs/SJR_CRISPRa_TF_lib_final.csv'
output_fasta = '/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final2.fasta'
guides = {}
with open(input_csv, 'r', newline='') as csvfile, open(output_fasta, 'w') as fastafile:
    reader = csv.DictReader(csvfile)
    reader.fieldnames = [field.strip().replace('\ufeff', '') for field in reader.fieldnames]
    for row in reader:
        gene = row['gene'].strip()
        spacer = row['protospacer'].strip()
        guides[gene] = spacer
        fastafile.write(f'>{gene}\n{spacer}\n')
# Parse FASTA
#guides = {}
#for record in SeqIO.parse(fasta_file, "fasta"):
#    guides[record.id] = str(record.seq)

# Parse SAM
metadata = {}
with open(sam_file, "r") as f:
    for line in f:
        if line.startswith("@"):
            continue
        parts = line.strip().split("\t")
        guide_id = parts[0]
        guide_chr = parts[2]
        guide_start = int(parts[3])  # 1-based
        flag = int(parts[1])
        strand = "-" if flag & 16 else "+"
        seq = parts[9]
        pam = seq[-3:] if len(seq) >= 3 else ""
        guide_end = guide_start + len(seq) - 1
        # Overwrite only if not already present (first alignment)
        if guide_id not in metadata:
            metadata[guide_id] = {
                "guide_id": guide_id,
                "spacer": guides.get(guide_id, ""),
                "targeting": "TRUE",
                "type": "targeting",
                "guide_chr": guide_chr,
                "guide_start": guide_start,
                "guide_end": guide_end,
                "strand": strand,
                "pam": pam,
                "genomic_element": "promoter",
                "intended_target_name": guide_id,  # Will be updated with Ensembl ID
                "intended_target_chr": guide_chr,
                "intended_target_start": "",
                "intended_target_end": "",
                "putative_target_genes": intended_target_name,
                "reporter": "",
                "imperfect": ""               
            }
# Function to get Ensembl ID from gene symbol
def get_ensembl_id(symbol):
    url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{symbol}?content-type=application/json"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        return data.get('id', '')
    else:
        print(f"Warning: Could not retrieve Ensembl ID for symbol '{symbol}'. HTTP Status: {response.status_code}")
        return ''
# Update intended_target_name with Ensembl IDs
for guide_id, info in metadata.items():
    gene_symbol = info['intended_target_name']
    if gene_symbol:
        ensembl_id = get_ensembl_id(gene_symbol)
        metadata[guide_id]['intended_target_name'] = ensembl_id
# Write TSV
headers = [
    "guide_id", "spacer", "targeting", "type", "guide_chr",
    "guide_start", "guide_end", "strand", "pam", "genomic_element",
    "intended_target_name", "intended_target_chr",
    "intended_target_start", "intended_target_end", "putative_target_genes", "reporter", "imperfect"
]
with open(output_file, "w", newline="") as out_f:
    writer = csv.DictWriter(out_f, fieldnames=headers, delimiter="\t")
    writer.writeheader()
    for guide_id in guides:
        row = metadata.get(guide_id, {
            "guide_id": guide_id,
            "spacer": guides[guide_id],
            "targeting": "TRUE",
            "type": "targeting",
            "guide_chr": "",
            "guide_start": "",
            "guide_end": "",
            "strand": "",
            "pam": "",
            "genomic_element": "",
            "intended_target_name": "",  # Will be updated with Ensembl ID
            "intended_target_chr": "",
            "intended_target_start": "",
            "intended_target_end": "",
            "putative_target_genes": "",
            "reporter": "",
            "imperfect": ""
        })
        writer.writerow(row)

Overwriting /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_metadata.py


In [6]:
%%writefile /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_2_metadata.py

import sys
import csv
import requests
import time
sam_file = sys.argv[1]
output_file = sys.argv[2]

# Input files
input_csv = "/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final.csv"         # Your input CSV
sam_file = "/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final.sam"         # Output from Bowtie
tss_file = "/work/rr151/SamAIM2neuro_data/gencode_v47_tss.bed"            # TSS BED file
output_file = "/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final2_metadata.tsv"    # Final annotated output
output_fasta = '/work/rr151/SamAIM2neuro_data/SJR_CRISPRa_TF_lib_final2.fasta'
# Step 1: Read protospacers from CSV
guides = {}  # key = guide ID (gene name), value = spacer sequence
guide_symbols = set()
with open(input_csv, 'r', newline='') as csvfile, open(output_fasta, 'w') as fastafile:
    reader = csv.DictReader(csvfile)
    reader.fieldnames = [field.strip().replace('\ufeff', '') for field in reader.fieldnames]
    for row in reader:
        gene = row["gene"].strip()
        spacer = row["protospacer"].strip()
        guides[gene] = spacer
        guide_symbols.add(gene.upper())
#       guides[gene] = spacer
        fastafile.write(f'>{gene}\n{spacer}\n')
# Step 2: Load TSS info
#tss_lookup = {}  #  key = gene symbol, value = (chr, start, end, Ensembl ID)
#with open(tss_file, "r") as f:
#    for line in f:
#        parts = line.strip().split("\t")
#       if len(parts) < 6:
#            continue
#        chrom, start, end, ensembl_id, gene_symbol, strand = parts
#        tss_lookup[gene_symbol.upper()] = (chrom, int(start), int(end), ensembl_id)

# Step 2: Map gene symbols to Ensembl IDs using Ensembl REST API
symbol_to_ensembl = {}
for symbol in guide_symbols:
    url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{symbol}?content-type=application/json"
    try:
        response = requests.get(url)
        if response.ok:
            data = response.json()
            ensembl_id = data.get("id", "")
            if ensembl_id:
                symbol_to_ensembl[symbol] = ensembl_id
        else:
            print(f"[Warning] Could not find Ensembl ID for {symbol} (HTTP {response.status_code})")
    except Exception as e:
        print(f"[Error] API call failed for {symbol}: {e}")
    time.sleep(0.1)  # Avoid rate limit
# Step 3: Load TSS BED into dict keyed by Ensembl ID
tss_lookup = {}  # ensembl_id → (chr, start, end)
with open(tss_file, "r") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 4:
            continue
        chrom, start, end, ensembl_id = parts[:4]
        tss_lookup[ensembl_id] = (chrom, int(start), int(end))

# Step 4: Parse SAM file
metadata = {}
with open(sam_file, "r") as f:
    for line in f:
        if line.startswith("@"):
            continue
        parts = line.strip().split("\t")
        guide_id = parts[0]
        guide_chr = parts[2]
        guide_start = int(parts[3])
        flag = int(parts[1])
        strand = "-" if flag & 16 else "+"
        seq = parts[9]
        pam = seq[-3:] if len(seq) >= 3 else ""
        guide_end = guide_start + len(seq) - 1
        gene_symbol = guide_id.rsplit(".",1)[0].upper()
        ensembl_id = symbol_to_ensembl.get(gene_symbol, "")
        tss = tss_lookup.get(ensembl_id, ("", "", ""))
        metadata[guide_id] = {
            "guide_id": guide_id,
            "spacer": guides.get(guide_id, ""),
            "targeting": "TRUE",
            "type": "targeting",
            "guide_chr": guide_chr,
            "guide_start": guide_start,
            "guide_end": guide_end,
            "strand": strand,
            "pam": pam,
            "genomic_element": "tss",
            "intended_target_name": ensembl_id,  # Ensembl ID from BED
            "intended_target_chr": tss[0],
            "intended_target_start": tss[1],
            "intended_target_end": tss[2],
            "putative_target_genes": gene_symbol,            
            "reporter": "",
            "imperfect": ""
        }
        
        
# Step 5: Write output TSV
headers = [
    "guide_id", "spacer", "targeting", "type", "guide_chr",
    "guide_start", "guide_end", "strand", "pam", "genomic_element",
    "intended_target_name", "intended_target_chr", "intended_target_start", "intended_target_end", "putative_target_genes", "reporter", "imperfect"
]
with open(output_file, "w", newline="") as out_f:
    writer = csv.DictWriter(out_f, fieldnames=headers, delimiter="\t")
    writer.writeheader()
    for guide_id in guides:
        row = metadata.get(guide_id)
        if row:
            writer.writerow(row)
        else:
            # fallback if not aligned in SAM
            gene_symbol = guide_id.rsplit(".",1)[0].upper()
            ensembl_id = symbol_to_ensembl.get(gene_symbol, "")
            tss = tss_lookup.get(ensembl_id, ("", "", ""))
            row = {
                "guide_id": guide_id,
                "spacer": guides[guide_id],
                "targeting": "TRUE",
                "type": "targeting",
                "guide_chr": "",
                "guide_start": "",
                "guide_end": "",
                "strand": "",
                "pam": "",
                "genomic_element": "tss", 
                "intended_target_name": ensembl_id,
                "intended_target_chr": tss[0],
                "intended_target_start": tss[1],
                "intended_target_end": tss[2],
                "putative_target_genes": gene_symbol,
                "reporter": "",
                "imperfect": ""
        }
        writer.writerow(row)


Overwriting /hpc/home/rr151/SAM_Neuro_data/parse_alignment_to_2_metadata.py


In [ ]:
# Step 2: Load TSS info
tss_lookup = {}  #  key = gene symbol, value = (chr, start, end, Ensembl ID)
with open(tss_file, "r") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 6:
            continue
        chrom, start, end, ensembl_id, gene_symbol, strand = parts
        tss_lookup[gene_symbol.upper()] = (chrom, int(start), int(end), ensembl_id)

In [5]:
%%writefile /hpc/home/rr151/SAM_Neuro_data/parse_alignment_with_gtf.py
import sys
import csv
# Input/output files
input_csv = "/work/rr151/SamAIM2neuro_data/subtype_lib.csv"
sam_file = "/work/rr151/SamAIM2neuro_data/SJR_subtype_lib_final.sam"
gtf_file = "/work/rr151/SamAIM2neuro_data/gencode.v47.annotation.gtf"
output_file = "/work/rr151/SamAIM2neuro_data/SJR_subtype_lib_final_metadata.tsv"
output_fasta = '/work/rr151/SamAIM2neuro_data/SJR_subtype_lib_final.fasta'
# Step 1: Load guide sequences
guides = {}
with open(input_csv, 'r', newline='') as csvfile, open(output_fasta, 'w') as fastafile:
    reader = csv.DictReader(csvfile)
    reader.fieldnames = [field.strip().replace('\ufeff', '') for field in reader.fieldnames]
    for row in reader:
        gene = row["gene"].strip()
        spacer = row["protospacer"].strip()
        guides[gene] = spacer
        fastafile.write(f'>{gene}\n{spacer}\n')
# Step 2: Parse GTF file to build TSS lookup
tss_lookup = {}  # key = gene_name.upper(), value = (chr, tss_start, tss_end, gene_id)
with open(gtf_file, "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        parts = line.strip().split("\t")
        if len(parts) < 9:
            continue
        chrom, _, feature_type, start, end, _, strand, _, attributes = parts
        if feature_type != "gene":
            continue
        # Extract gene_id and gene_name from attributes field
        attr_dict = {}
        for attr in attributes.strip().split(";"):
            if attr.strip():
                key_value = attr.strip().split(" ", 1)
                if len(key_value) == 2:
                    key, value = key_value
                    attr_dict[key] = value.replace('"', '').strip()
        gene_id = attr_dict.get("gene_id", "")
        gene_name = attr_dict.get("gene_name", "").upper()
        if not gene_id or not gene_name:
            continue
        # Determine TSS
        start, end = int(start), int(end)
        if strand == "+":
            tss_start, tss_end = start, start + 1
        else:
            tss_start, tss_end = end - 1, end
        tss_lookup[gene_name] = (chrom, tss_start, tss_end, gene_id)
# Step 3: Parse SAM file and build metadata
metadata = {}
with open(sam_file, "r") as f:
    for line in f:
        if line.startswith("@"):
            continue
        parts = line.strip().split("\t")
        guide_id = parts[0]
        guide_chr = parts[2]
        guide_start = int(parts[3])
        flag = int(parts[1])
        strand = "-" if flag & 16 else "+"
        seq = parts[9]
        pam = seq[-3:] if len(seq) >= 3 else ""
        guide_end = guide_start + len(seq) - 1
        gene_symbol = guide_id.rsplit('.', 1)[0].upper()
        tss = tss_lookup.get(gene_symbol, ("", "", "", ""))
        metadata[guide_id] = {
            "guide_id": guide_id,
            "spacer": guides.get(guide_id, ""),
            "targeting": "TRUE",
            "type": "targeting",
            "guide_chr": guide_chr,
            "guide_start": guide_start,
            "guide_end": guide_end,
            "strand": strand,
            "pam": pam,
            "genomic_element": "tss",
            "intended_target_name": tss[3],  # Ensembl gene ID
            "intended_target_chr": tss[0],
            "intended_target_start": tss[1],
            "intended_target_end": tss[2],
            "putative_target_genes": "",
            "reporter": "",
            "imperfect": ""
        }
# Step 4: Write output
headers = [
    "guide_id", "spacer", "targeting", "type", "guide_chr",
    "guide_start", "guide_end", "strand", "pam", "genomic_element",
    "intended_target_name", "intended_target_chr", "intended_target_start",
    "intended_target_end", "putative_target_genes", "reporter", "imperfect"
]
with open(output_file, "w", newline="") as out_f:
    writer = csv.DictWriter(out_f, fieldnames=headers, delimiter="\t")
    writer.writeheader()
    for guide_id in guides:
        gene_symbol = guide_id.rsplit('.', 1)[0].upper()
        tss = tss_lookup.get(gene_symbol, ("", "", "", ""))
        row = metadata.get(guide_id, {
            "guide_id": guide_id,
            "spacer": guides[guide_id],
            "targeting": "TRUE",
            "type": "targeting",
            "guide_chr": "",
            "guide_start": "",
            "guide_end": "",
            "strand": "",
            "pam": "",
            "genomic_element": "tss",
            "intended_target_name": tss[3],
            "intended_target_chr": tss[0],
            "intended_target_start": tss[1],
            "intended_target_end": tss[2],
            "putative_target_genes": "",
            "reporter": "",
            "imperfect": ""
        })
        writer.writerow(row)

Overwriting /hpc/home/rr151/SAM_Neuro_data/parse_alignment_with_gtf.py
